In [10]:
from pre_data_f import pre_data
from cheek_pred_effect import construct_test_loader_by_dic

from model_ import PosEmb_Seq2eq
# from utils import count_parameters
# from utils import pad_list

from cheek_pred_effect import generate_pred_confs 
from utils_base import pickle_
from utils_base import info
from utils_base import repeat_list_by_numlist
from utils_base import group_by_num
from utils_base import reorder

from utils_others import extract_mol_from_chemdir
from utils_others import rotate_a_mol
from utils_others import writosdf


from utils import count_parameters

from loops import batch_inference, inference

import torch
from torch import nn
from torch.optim import Adam
import copy
import os.path as osp
from tqdm.notebook import tqdm

# from dgllife.utils import Meter
# from functools import partial

import numpy as np
import time

# torch.set_printoptions(profile="full")  
torch.set_printoptions(sci_mode=False) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [11]:
path = "./data_1"
ol = "drugs"
nam = "1088"

In [12]:
chem_dataset_dir = "data_1/rdkit_folder/drugs"

test_dic_path = f"./prepare_ori_con/test_{nam}_dic.pkl"

smiles_list_path = osp.join(path, ol, "smiles_list.pkl" )          
torsion_idx_list_path = osp.join(path, ol, "torsion_idx_list.pkl")

torsion_list_path = osp.join(path, ol, "torsion_list.pkl")
relativeenergy_list_path = osp.join(path, ol, "relativeenergy_list.pkl")

batch_size = 16

max_Emb_wl = 40    
max_Emb_d = 10     

# generate feat
max_num_atom=90
wl_max_iter=3
num_choosed_confs = 12000


rate_trte=0.9
rate_tr_in_trti=0.8


print_every = 100
model_save_every = 10000

# epochs
num_epochs = 100

In [13]:
model = PosEmb_Seq2eq(          # 4 * (node_feat_dim + outp_dim*3) + 3 * edge_feat_dim 

    pf_dim_pos=32,
    pf_dim1=32,  # I take 256
    pf_dim2=32,  # I take 256
    max_Emb_wl=max_Emb_wl,   
    max_Emb_d=max_Emb_d,

    # pf_dim_readout=256,  # I take 256
    outp_dim=16,  # I take 16
    trg_emb_dim=32,

    ENC_LAYERS=1,
    ENC_HEADS=5, 
    ENC_PF_DIM=32,
    DEC_LAYERS=3,
    DEC_HEADS=4,
    DEC_PF_DIM=32,
    device=device,
    
    t_e_ENC_LAYERS=1,
    t_e_ENC_HEADS=5,   
    t_e_ENC_PF_DIM=32,
    
    noise_dim = 100,
    noise_loc=0, 
    noise_scale=5,
    
    edge_feat_dim=10,
    node_feat_dim=39,
    dropout=0.1,
    max_num_atom=max_num_atom
).to(device)

In [14]:
loss_criterion = nn.MSELoss()   # ignore_index = TRG_PAD_IDX  
# loss_criterionn = nn.MSELoss()   # ignore_index = TRG_PAD_IDX  

loss_criterion_= nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 0.0005,
                 weight_decay=10 ** -5
                )

In [16]:
print("starting......")
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
smilelist = pickle_.load(smiles_list_path)
torsion_angle = pickle_.load(torsion_list_path)
torsion_list = pickle_.load(torsion_idx_list_path)
relativeenergy_list = pickle_.load(relativeenergy_list_path)

# torsion_angle = [list(zip(torsion[0],(np.array(torsion[1])+1).tolist())) for torsion in torsion_angle]

print("done")
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) )
print(len(smilelist), len(torsion_angle), len(torsion_list), len(relativeenergy_list))
torsion_angle[0].device

starting......
2023-04-26 13:54:38
done
2023-04-26 13:55:12
185469 185469 185469 185469


device(type='cpu')

In [7]:
train_loader, val_loader, test_loader = pre_data(
    smilelist, torsion_list, torsion_angle, relativeenergy_list, batch_size,
    rate_trte, rate_tr_in_trti, max_num_atom, wl_max_iter, num_choosed_confs, 
    device = device
     )
print(len(train_loader),  len(val_loader), len(test_loader))

9274 1160 1160


In [17]:
test_loader = construct_test_loader_by_dic(test_dic_path, 
                                            smilelist,
                                            torsion_angle,
                                            torsion_list,
                                            relativeenergy_list,
                                            batch_size,
                                            max_num_atom, 
                                            wl_max_iter, 
                                            num_choosed_confs, 
                                            device = device)


/home/gpu2-data1/zhangzimei/work_now/Tora3D/Tora3D/cheek_pred_effect.py:59: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  torsion_angle_200 = list(np.array([t.cpu() for t in torsion_angle])[idxs])
/home/gpu2-data1/zhangzimei/work_now/Tora3D/Tora3D/cheek_pred_effect.py:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  torsion_angle_200 = list(np.array([t.cpu() for t 

len(test_loader) 68


/home/gpu2-data1/zhangzimei/work_now/Tora3D/Tora3D/cheek_pred_effect.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  relativeenergy_list_200 = list(np.array(relativeenergy_list)[idxs])


In [18]:
#     for batch_id, batch_data in enumerate(train_loader):
#         atom_feat, AM_bond_feat, node_color_feat, d, labels, masks, relativeenergys, num_atom_list, angle_group_num, num_confs_list, smiles_list = batch_data
#         break

In [19]:
def count_orderM(prediction,labels, num_confs_list,angle_group_num):

    prediction_t = torch.cat([prediction[:,:,0:1], prediction[:,:,1:].argmax(-1).unsqueeze(-1)-1],-1)    
    pred_angles = (prediction_t[:,:,0]+0.5)*180*prediction_t[:,:,1]
    true_angles = (labels[:,:,0]+0.5)*180*labels[:,:,1]


    prediction_tense = group_by_num(pred_angles, num_confs_list)
    labels_tense = group_by_num(true_angles, num_confs_list)
    prediction_tense_ = [prediction_tense[index][:,:group].detach().cpu().numpy() for index,group in enumerate(angle_group_num)]
    labels_tense_     = [labels_tense[index][:,:group].detach().cpu().numpy()     for index,group in enumerate(angle_group_num)]

    orderM_list = [torch.tensor(reorder(p,l)[2], dtype=torch.float32) for p,l in zip(prediction_tense_, labels_tense_)]
    return orderM_list

In [20]:
def run_a_train_epoch( epoch, model, data_loader, loss_criterion, loss_criterion_, optimizer):
    device = model.device
    model.train()
    covr_best = 0
    k = 0
    for batch_id, batch_data in enumerate(train_loader):
        atom_feat, AM_bond_feat, node_color_feat, d, labels, masks, relativeenergys, num_atom_list, angle_group_num, num_confs_list, smiles_list = batch_data
        batchsize = len(num_confs_list)
        
        prediction = model(device, 
        atom_feat, AM_bond_feat, node_color_feat, d, labels, masks, relativeenergys, num_atom_list, angle_group_num, num_confs_list, smiles_list)
        
        orderM_list = count_orderM(prediction,labels, num_confs_list,angle_group_num)

        prediction_tense = group_by_num(prediction, num_confs_list)
        labels_tense     = group_by_num(labels, num_confs_list)
        prediction_tense_ = [prediction_tense[index][:,:group] for index,group in enumerate(angle_group_num)]
        labels_tense_  = [torch.mm(orderM_list[index].to(device),
            labels_tense[index][:,:group].reshape(labels_tense[index].shape[0],-1)).reshape(labels_tense[index].shape[0],-1,2) 
                             for index,group in enumerate(angle_group_num)]
        loss = 0
        for i in range(batchsize):
            loss_value = loss_criterion(prediction_tense_[i][:,:,0],labels_tense_[i][:,:,0])
            loss_sig = loss_criterion_(prediction_tense_[i][:,:,1:].permute(0,2,1), 
                               (labels_tense_[i][:,:,1]+1).long())/10
            loss_ = loss_value+loss_sig
            loss+=loss_
        loss = loss/batchsize
      
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_id % print_every == 0:
            print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
            
            print('epoch {:d}/{:d}, batch {:d}/{:d}, loss {:.4f}'.format(
                epoch + 1, num_epochs, batch_id + 1, len(data_loader), loss.item()))
            print("loss_value:",loss_value.item())
            print("loss_sig:",loss_sig.item())
            print("loss:",loss.item())

            model.noise_scale = 100
            
            matr_,matp_,covr_,covp_,\
                lab_angles_allconfs_list_list,angles_allconfs_list_list= generate_pred_confs(model, test_loader, 
                                                                        loss_criterion, loss_criterion_, 
                                                                        save_conf_root="./confs_save",
                                                                        chem_dataset_dir = chem_dataset_dir,
                                                                        ori_confs_path  = test_dic_path,                                                                 
                                                                        optimizer=None, 
                                                                        debug = False,
                                                                        angle_range_list = [(-20,20),(40,100)],
                                                                        k = "2",
                                                                        max_re = 6,
                                                                        kn = 2,
                                                                        f_percent=0.25,
                                                                        part_batch=(0,6)
                                                                          )
            print("epoch:", epoch, "   covr_:",covr_)
            k+=1
            if covr_ > covr_best:
                covr_best = covr_
                batch_id_best = batch_id
                model_best = model
                k = 0
                print(f"so far the best is {covr_best}")
            if k == 10:
                print(f"{k} times no more improvement， So far，the best is: {batch_id_best}/{len(data_loader)} _ {covr_best}")
                break
        if batch_id % model_save_every == 0:
            print("saved")
    return batch_id_best,covr_best,model_best

In [21]:
def run_an_eval_epoch(epoch, model, data_loader, loss_criterion, loss_criterion_, optimizer=None):

    model.eval()
    loss_values_list = []
    loss_siges_list = []
    losses_list = []
    acc_sig_list = []
#     Labels = []
#     Prediction = []
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            atom_feat, AM_bond_feat, node_color_feat, d, labels, masks, relativeenergys, num_atom_list, angle_group_num, num_confs_list = batch_data

            prediction = model(device, atom_feat, AM_bond_feat, node_color_feat, d, labels, masks, relativeenergys, num_atom_list, angle_group_num, num_confs_list)

            prediction_ = torch.cat([prediction[index][0: group] for index,group in enumerate(repeat_list_by_numlist(angle_group_num,num_confs_list))])
            labels_ = torch.cat([labels[index][0: group] for index, group in enumerate(repeat_list_by_numlist(angle_group_num,num_confs_list))])        

            loss_value = loss_criterion(prediction_[:,0], labels_[:,0]).item()
            loss_sig=loss_criterion_(prediction_[:,1:], (labels_[:,1]+1).long()).item()/10
            loss = loss_value+loss_sig
            acc_sig = ((prediction_[:,1:].argmax(-1)-1 == labels_[:,1]).sum()/len(labels_)).item()
            
            loss_values_list.append(loss_value)
            loss_siges_list.append(loss_sig)
            losses_list.append(loss)        
            acc_sig_list.append(acc_sig)
            
#             if batch_id == 100:
#                 break
    loss_value = np.mean(loss_values_list)
    loss_sig = np.mean(loss_siges_list)
    loss = np.mean(losses_list)
    acc_sig = np.mean(acc_sig_list)
    
    return loss_value, loss_sig, loss, acc_sig 

In [23]:
# load model
cheekpoint = torch.load('model_save/model_save_main_drugs-Copy3ot_2022-06-14_20h10m42s.pth')
model.load_state_dict(cheekpoint['model'])


<All keys matched successfully>

### for higher score

In [14]:
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
model.noise_scale = 100

matr_,matp_,covr_,covp_,df,\
lab_angles_allconfs_list_list,angles_allconfs_list_list,_list,\
smiles_list,num_confs_list_list,num_atom_list_list,angle_group_num_list= generate_pred_confs(model, test_loader, 
                                                            loss_criterion, loss_criterion_, 
                                                            # save_conf_root="./confs_save",
                                                            chem_dataset_dir = chem_dataset_dir,
                                                            ori_confs_path  = test_dic_path,                                                                 
                                                            optimizer=None, 
                                                            debug = False,
                                                            angle_range_list = [(-20,20),(40,100)],
                                                            k = "2",
                                                            max_re = 6,
                                                            kn = 2,
                                                            f_percent=0.25,
                                                            # part_batch = [0,20]
                                                                                )


time_save = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print(time_save)
pickle_.save(f"./result_save/result_{nam}_{time_save}.pkl", (matr_,matp_,covr_,covp_,df))
print(matr_,matp_,covr_,covp_)

### for more reasonable structure

In [25]:
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
model.noise_scale = 100

matr_,matp_,covr_,covp_,df,\
lab_angles_allconfs_list_list,angles_allconfs_list_list,_list,\
smiles_list,num_confs_list_list,num_atom_list_list,angle_group_num_list= generate_pred_confs(model, test_loader, 
                                                            loss_criterion, loss_criterion_, 
                                                            save_conf_root="./confs_save_",
                                                            chem_dataset_dir = chem_dataset_dir,
                                                            ori_confs_path  = test_dic_path,                                                                 
                                                            optimizer=None, 
                                                            debug = False,
                                                            # angle_range_list = [(-20,20),(40,100)],
                                                            k = "2",
                                                            max_re = 6,
                                                            kn = 2,
                                                            f_percent=0.25,
                                                            # part_batch = [0,20]
                                                                                )


time_save = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print(time_save)
pickle_.save(f"./result_save/result_{nam}_{time_save}.pkl", (matr_,matp_,covr_,covp_,df))

In [ ]:
# model save
time_save=time.strftime("%Y-%m-%d_%Hh%Mm%Ss", time.localtime())
torch.save({
#                 'epoch': len(Train_loss),
                'model': model.state_dict(),
                'model_optimizer': optimizer.state_dict(),

#                 "Train_loss_value": Train_loss_value,
#                 "Train_loss_sig": Train_loss_sig,
#                 "Train_loss": Train_loss,
#                 "Train_acc_sig": Train_acc_sig,
                
#                 "Val_loss_value": Val_loss_value,
#                 "Val_loss_sig": Val_loss_sig,
#                 "Val_loss": Val_loss,
#                 "Val_acc_sig": Val_acc_sig,
    
},
          f"model_save/model_save_main_drugs-Copy3ot_{time_save}.pth")

print("save model to", f"model_save/model_save_main_drugs-Copy3ot_{time_save}.pth")